In [0]:
%sql
-- Remove o `%sql` se estiver usando no SQL Editor
WITH cleaned_cards AS (
    SELECT
        *,
        -- Limpeza do campo manaCost
        CASE 
            WHEN manaCost IS NULL OR manaCost IN ('', 'null') THEN '{0}'
            ELSE TRIM(manaCost)
        END AS cleaned_manaCost,

        -- Renomeando 'text' para evitar conflito com palavra reservada
        text AS card_text
    FROM magic_the_gathering.bronze.cards
)

SELECT 
    -- Identificação da carta
    id AS ID_CARD,
    INITCAP(name) AS NME_CARD,

    -- Descrição do custo de mana
    cleaned_manaCost AS DESC_MANA_COST,

    -- Custo convertido (CMC)
    (
        COALESCE(
            CAST(
                REGEXP_REPLACE(
                    REGEXP_REPLACE(
                        cleaned_manaCost,
                        '[^0-9{}]', ''
                    ),
                    '[{}]', ''
                ) AS INT
            ), 0
        )
        +
        LENGTH(REGEXP_REPLACE(
            cleaned_manaCost,
            '[^WUBRGC]', ''
        ))
    ) AS MANA_COST,

    -- Tipo da carta (dividido)
    CASE
        WHEN LOWER(type) LIKE '%planeswalker%' THEN 'Planeswalker'
        WHEN type LIKE '%—%' THEN TRIM(SPLIT(type, '—')[0])
        ELSE TRIM(type)
    END AS NME_CARD_TYPE,

    CASE
        WHEN LOWER(type) LIKE '%planeswalker%' THEN type
        WHEN type LIKE '%—%' THEN TRIM(SPLIT(type, '—')[1])
        ELSE 'NA'
    END AS DESC_CARD_TYPE,

    INITCAP(rarity) AS NME_RARITY,

    -- Cores da carta (formatado)
    CASE 
        WHEN colors IS NULL OR colors IN ('', '[]') THEN 'Colorless'
        ELSE REPLACE(REPLACE(REPLACE(REPLACE(colors, '[', ''), ']', ''), '"', ''), ' ', '')
    END AS COD_COLORS,

    -- Categoria de cor (Colorless, Mono, Dual, Multicolor)
    CASE
        WHEN colors IS NULL OR colors IN ('', '[]') THEN 'Colorless'
        WHEN LENGTH(REGEXP_REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(colors, '[', ''), ']', ''), '"', ''), ' ', ''), '[^,]', '')) = 0 THEN 'Mono'
        WHEN LENGTH(REGEXP_REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(colors, '[', ''), ']', ''), '"', ''), ' ', ''), '[^,]', '')) = 1 THEN 'Dual Color'
        WHEN LENGTH(REGEXP_REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(colors, '[', ''), ']', ''), '"', ''), ' ', ''), '[^,]', '')) >= 2 THEN 'Multicolor'
        ELSE 'Mono'
    END AS NME_COLOR_CATEGORY,

    -- Quantidade de cores (por símbolos no custo)
    CASE
        WHEN cleaned_manaCost IN ('{0}', '1') THEN 0
        WHEN REGEXP_LIKE(cleaned_manaCost, '^[0-9X]+$') THEN 0
        ELSE LENGTH(REGEXP_REPLACE(UPPER(cleaned_manaCost), '[^WUBRG]', ''))
    END AS QTY_COLORS,

    -- Estatísticas da criatura
    COALESCE(CAST(power AS INT), 0) AS NME_POWER,
    COALESCE(CAST(toughness AS INT), 0) AS NME_TOUGHNESS,

    -- Informações de edição
    UPPER(set) AS COD_SET,
    INITCAP(artist) AS NME_ARTIST,

    -- Texto da carta (tratado)
    CASE
        WHEN card_text IS NULL OR card_text = 'null' THEN 'NA'
        ELSE TRIM(
            translate(
                REGEXP_REPLACE(
                    REGEXP_REPLACE(
                        REGEXP_REPLACE(
                            REGEXP_REPLACE(
                                REGEXP_REPLACE(
                                    REGEXP_REPLACE(
                                        REGEXP_REPLACE(
                                            REGEXP_REPLACE(
                                                REGEXP_REPLACE(
                                                    REGEXP_REPLACE(
                                                        REGEXP_REPLACE(
                                                            REGEXP_REPLACE(
                                                                card_text, '\\n', ' '), -- quebra de linha
                                                            '""', '"'),
                                                        '\{W\}', '[White]'),
                                                    '\{U\}', '[Blue]'),
                                                '\{B\}', '[Black]'),
                                            '\{R\}', '[Red]'),
                                        '\{G\}', '[Green]'),
                                    '\{C\}', '[Colorless]'),
                                '\{X\}', '[X]'),
                            '\{T\}', '[Tap]'),
                        '\{Q\}', '[Untap]'),
                    '\{S\}', '[Snow]'),
                '\{E\}', '[Energy]'),
            '\{0\}', '[0]'),
            '\{1\}', '[1]'),
            'áàãâäéèêëíìîïóòõôöúùûüçÁÀÃÂÄÉÈÊËÍÌÎÏÓÒÕÔÖÚÙÛÜÇ',
            'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC'
        )
    END AS DESC_CARD,

    -- Hash do texto (chave de dimensão de habilidades)
    CASE
        WHEN card_text IS NULL OR card_text IN ('null', '') THEN 'NA'
        ELSE MD5(LOWER(TRIM(card_text)))
    END AS COD_DIM_HABILITY

FROM cleaned_cards;
